In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

customers = pd.read_csv("C:/Users/dell/Downloads/Customerss.csv")
products = pd.read_csv("C:/Users/dell/Downloads/Products.csv")
transactions = pd.read_csv("C:/Users/dell/Downloads/Transactions.csv")

merged_data = transactions.merge(customers, on='CustomerID').merge(products, on='ProductID')

if 'Price_x' in merged_data.columns:
    merged_data.rename(columns={'Price_x': 'Price'}, inplace=True)
elif 'Price_y' in merged_data.columns:
    merged_data.rename(columns={'Price_y': 'Price'}, inplace=True)

customer_features = merged_data.groupby("CustomerID").agg({
    "TotalValue": "sum",
    "TransactionID": "count",
    "Price": "mean"
}).rename(columns={
    "TotalValue": "TotalSpent",
    "TransactionID": "TransactionCount",
    "Price": "AvgPrice"
})

similarity_matrix = cosine_similarity(customer_features)

similarity_df = pd.DataFrame(similarity_matrix, index=customer_features.index, columns=customer_features.index)

lookalike_dict = {}
for cust_id in customer_features.index[:20]: 
    similar_customers = similarity_df[cust_id].nlargest(4).iloc[1:4]  
    lookalike_dict[cust_id] = list(zip(similar_customers.index, similar_customers.values))

lookalike_df = pd.DataFrame.from_dict(lookalike_dict, orient='index')
lookalike_df.to_csv("ShrajnaR_Lookalike.csv", index=True)
